<a href="https://colab.research.google.com/github/quanap5/ColabNote2/blob/master/Computing_Gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


### Computing gradient

In [11]:
import tensorflow as tf

x = tf.placeholder(tf.float32)
# y =  2*x*x
z_1 = tf.multiply(tf.constant(2.0), x)
z_2 = tf.multiply(z_1, x)
var_grad = tf.gradients(z_2, x)

with tf.Session() as session:
  var_grad_val = session.run(var_grad,feed_dict={x:2})
  print(var_grad_val)

[8.0]


### 1.1 The following is my function which has explicit equation


In [0]:
def function_1(x):
  return tf.sin(x)*x + tf.cos(x) * tf.exp(x)

### 1.2 Then, I use following two functions to compute the gradient and obtain  the same results as expected

In [31]:
# method 1
x = tf.placeholder(tf.float32)
gradient_1 = tf.gradients(function_1(x), x)
with tf.Session() as session:
  gradient_1_val = session.run(gradient_1, feed_dict={x: 0.003})
  print(gradient_1_val)

[1.005991]


In [32]:
# method 2
x = tf.placeholder(tf.float32)
gradient_2 = tf.train.AdamOptimizer().compute_gradients(function_1(x), var_list=x)
with tf.Session() as session:
  gradient_1_val = session.run(gradient_2, feed_dict={x: 0.003})
  print(gradient_1_val) # note that: compute_gradient also return itself value of x "array(x, dtype=float32)"

[(1.005991, array(0.003, dtype=float32))]


### 1.3 Another example

In [33]:
import tensorflow as tf
import numpy as np


def expectation(x):
    point = np.reshape(np.linspace(-5.0, 5.0, 300), (300, 1))
    kernel_matrix_np = np.exp(-(point - np.transpose(point))**2 / (2 * 1.5**2))

    def reference_point():
        covariance = kernel_matrix_np
        np.random.seed(100)
        sampled_funcs = np.random.multivariate_normal(np.ones(len(point)), covariance)
        return sampled_funcs
      
      
    ref_point = tf.transpose(tf.convert_to_tensor(reference_point(), dtype=tf.float32))  # <---
    five64 = tf.convert_to_tensor(5.0, dtype=tf.float32)  # <---
    point = tf.reshape(tf.linspace(-five64, 5.0, 300), (300, 1))  # <---
    kernel_matrix_tf = tf.exp(-(point - tf.transpose(point))**2 / (2 * 1.5**2))
    inverse_kernel_matrix = tf.matrix_inverse(kernel_matrix_tf)
    kernel_vector = tf.exp(-(x - tf.transpose(point))**2 / (2 * 1.5**2))
    mu = tf.matmul(kernel_vector, inverse_kernel_matrix)
    ref_point = tf.expand_dims(ref_point, axis=1)
    mu = tf.matmul(mu, ref_point)
    return mu

x = tf.Variable(2.0, dtype=tf.float32, name='x')  # <---
gradients_1 = tf.train.AdamOptimizer().compute_gradients(expectation(x), var_list=[x])
graidents_2 = tf.gradients(expectation(x), [x])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('The gradient one is', sess.run(gradients_1))
    print('The gradient two is', sess.run(graidents_2))

The gradient one is [(-25.512436, 2.0)]
The gradient two is [-25.512436]
